In [ ]:
import pandas as pd
import json
from glob import glob
from pathlib import Path
from types import SimpleNamespace
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 12)

sns.set_context('poster')

dir_root = Path('').absolute()
dir_calc_zora = dir_root.joinpath('calcs_mw_zora')

In [ ]:
def parseFileName(f):
    mol, kin, take, c, ext = f.name.split('_')
    return mol, kin, take, float(c)

In [ ]:
def loadFile(f):
    with open(f) as file:
        job = json.loads(file.read(), object_hook=lambda item: SimpleNamespace(**item))
        
    try:
        job.output
    except AttributeError:
        job.output = SimpleNamespace()
        
    return job

In [ ]:
def success(job):
    try:
        return job.output.success
    except AttributeError:
        return False

In [ ]:
table = []
for f in dir_calc_zora.glob('*.json'):
    mol, kin, take, c = parseFileName(f)
    
    job = loadFile(f)
    ok = success(job)
    
    prec = job.input.scf_calculation.scf_solver.final_prec
    c = job.input.scf_calculation.scf_solver.light_speed
    e = job.output.properties.scf_energy.E_tot if ok else None
    niter = len(job.output.scf_calculation.scf_solver.cycles) if ok else None
    w = job.output.scf_calculation.scf_solver.wall_time / niter * job.output.provenance.total_cores if ok else None
    final_update = job.output.scf_calculation.scf_solver.cycles[-1].energy_update if ok else None
    
    table.append((mol, kin, take, c, e, niter, prec, w, final_update))
    
df = pd.DataFrame(table, columns=['Molecule', 'Kinetic', 'Take', 'LightSpeed', 'Energy', 'NIter', 'Precision', 'CPUTime', 'EnergyConvergence']).sort_values(by=['LightSpeed', 'Molecule', 'Kinetic', 'Take']).reset_index(drop=True)
df['Algorithm'] = df.Kinetic + df.Take

In [ ]:
print(df.loc[(df.LightSpeed.map(int) == 13) & (df.Kinetic.map(int) == 0)].sort_values(by=['Molecule', 'Take', 'Kinetic'])[['Molecule', 'Take', 'Kinetic', 'LightSpeed', 'Precision', 'Energy', 'EnergyConvergence']])

In [ ]:
print(df.loc[(df.LightSpeed.map(int) == 137) & (df.Kinetic.map(int) == 0)].sort_values(by=['Molecule', 'Take', 'Kinetic'])[['Molecule', 'Take', 'Kinetic', 'LightSpeed', 'Precision', 'Energy', 'EnergyConvergence']])

In [ ]:
print(df.loc[(df.LightSpeed > 138) & (df.Kinetic.map(int) == 0)].sort_values(by=['Molecule', 'Take', 'Kinetic'])[['Molecule', 'Take', 'Kinetic', 'LightSpeed', 'Precision', 'Energy', 'EnergyConvergence']])

In [ ]:
print(df.pivot(index='Molecule', values=['EnergyConvergence'], columns=['Take', 'Kinetic', 'LightSpeed']).transpose().sort_values(by=['Kinetic', 'Take', 'LightSpeed']))

In [26]:
table = []
d_zora = Path('').absolute().joinpath('calcs_mw_zora/')
d_nr = Path('').absolute().joinpath('calcs_mw_nr/')

for mol in ['He', 'Ne', 'Ar', 'Kr', 'H2O', 'LiH']:
    zora = loadFile(d_zora.joinpath(f'{mol}_0_0_137036_.json'))
    nr = loadFile(d_nr.joinpath(mol+'_.json'))
    
    e_zora = zora.output.properties.scf_energy.E_tot
    e_nr = nr.output.properties.scf_energy.E_tot
    
    print(mol, e_zora, e_nr, abs(e_zora - e_nr))

He -2.892935491418813 -2.892935491135128 2.8368463134142985e-10
Ne -93.79226382987301 -93.79226358279418 2.4707883028440847e-07
Ar -312.6875153796463 -312.6875127465987 2.6330475861868763e-06
Kr -1273.201397756162 -1273.201353956275 4.379988695291104e-05
H2O -76.38852580283374 -76.38852568901055 1.1382319087260839e-07
LiH -8.047347899171163 -8.04734789762583 1.5453327506520509e-09
